In [1]:
import yfinance as yf
import talib
import numpy as np
from package import indicator as ind
import pandas as pd
from pandas_datareader import data as pdr
import random
from sklearn.decomposition import PCA
import sklearn.metrics
from genetic_selection import GeneticSelectionCV
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import datetime as datetime

import warnings
from package import euklid_regressor as eur
warnings.filterwarnings("ignore")

random.seed(42)
stock = "IBM"

In [2]:
#import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)
# download dataframe
start = datetime.datetime(2000,8,23)
end = datetime.datetime(2022,4,1)
data = pdr.get_data_yahoo("IBM", start=start, end=end)


[*********************100%***********************]  1 of 1 completed


In [3]:
data['vwap'] = eur.vwap(data['Close'], data['Volume'])
data['SO'] = eur.Stochastic_Oscillator(data['Close'])
data["ROC_1"] =  talib.ROC(data['Close'], timeperiod=1)
data["ROC_2"] =  talib.ROC(data['Close'], timeperiod=2)
data["ROC_3"] =  talib.ROC(data['Close'], timeperiod=3)
data["ROC_4"] =  talib.ROC(data['Close'], timeperiod=4)
data["ROC_5"] =  talib.ROC(data['Close'], timeperiod=5)
data["ROC_6"] =  talib.ROC(data['Close'], timeperiod=6)
data["ROC_7"] =  talib.ROC(data['Close'], timeperiod=7)
data["ROC_30"] =  talib.ROC(data['Close'], timeperiod=30)
data["ROC_60"] =  talib.ROC(data['Close'], timeperiod=60)
data['Momentum'] = ind.momentum(data)
data['Return'] = data.Close.pct_change()
data['1 Day ROI']=ind.ROI(data,1)
data['2 Day ROI']=ind.ROI(data,2)
data['3 Day ROI']=ind.ROI(data,3)
data['4 Day ROI']=ind.ROI(data,4)
data['5 Day ROI']=ind.ROI(data,5)
data['6 Day ROI']=ind.ROI(data,6)
data['7 Day ROI']=ind.ROI(data,7)
data['10 Day ROI']=ind.ROI(data,10)
data['20 Day ROI']=ind.ROI(data,20)
data['30 Day ROI']=ind.ROI(data,30)
data['RSI_1'] = eur.rsi(data['Close'],1)
data['RSI_2'] = eur.rsi(data['Close'],2)
data['RSI_3'] = eur.rsi(data['Close'],3)
data['RSI_4'] = eur.rsi(data['Close'],4)
data['RSI_5'] = eur.rsi(data['Close'],5)
data['RSI_6'] = eur.rsi(data['Close'],6)
data['RSI_7'] = eur.rsi(data['Close'],7)
data['RSI_14'] = eur.rsi(data['Close'],14)
data['RSI_30'] = eur.rsi(data['Close'],30)
data['RSI_60'] = eur.rsi(data['Close'],60)
data['EMA_12'] = ind.EMA(data, 12)
data['EMA_26'] = ind.EMA(data, 26)
data['Up down'] = ind.clas(data, .01)
eur.macd(data,data['Close'])
#Add SRSI to our dataframe
data['SRSI_10'] = ind.SRSI(data,10)
data['SRSI_14'] = ind.SRSI(data,14)
data['SRSI_30'] = ind.SRSI(data,30)
#data['Williams_1'] = ind.Williams(data,1)
#data['Williams_2'] = ind.Williams(data,2)
#data['Williams_3'] = ind.Williams(data,3)
data['Williams_14'] = ind.Williams(data,14)
data['ATR_14'] = ind.ATR(data,14) 
data['CCI'] = ind.CCI(data,20) 
#data['SAR'] = talib.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0)
#data['AROONOSC_7'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=7)
#data['AROONOSC_14'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=14)
#data['AROONOSC_21'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=21)
data["APO"] = talib.APO(data['Close'], fastperiod=12, slowperiod=26, matype=0)
differenced = (data['Close'] - data['Close'].shift(1))/(data['Close'].shift(1))[1:]
previous_differenced = differenced.shift(1)
data["Differenced"] = differenced
data["Previous_differenced"] = previous_differenced

data=data.drop(['Adj Close','Volume'], axis=1)
data = data.dropna()


# Cleaning

In [4]:
data = data.dropna()
data

,Open,High,Low,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,...,MACDs_12_26_9,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced
Date,,,,,,,,,,,,,,,,,,,,,
2000-11-15,95.124283,95.662048,93.331741,95.004784,93.401956,69.281105,-0.125624,1.988455,6.854843,-0.062848,...,-3.708187,0.539359,0.854175,0,-32.558094,4.172128,44.403379,0.086562,-0.001256,0.021167
2000-11-16,94.885277,95.303535,93.929253,93.929253,93.898056,56.375838,-1.132081,-1.256283,0.833863,5.645160,...,-3.469787,0.362031,0.769068,0,-40.930228,3.799082,36.549445,0.572234,-0.011321,-0.001256
2000-11-17,93.989006,97.992355,93.929253,97.454590,94.440657,95.973191,3.753184,2.578614,2.449750,4.618343,...,-3.190483,0.576667,0.898764,1,-14.948429,3.834285,67.459641,0.884780,0.037532,-0.011321
2000-11-20,97.036331,98.709366,96.438812,98.709366,95.171420,100.000000,1.287549,5.089057,3.899364,3.768841,...,-2.872892,0.613039,0.780316,0.322913,-4.123718,3.625787,84.436953,1.203071,0.012875,0.037532
2000-11-21,98.709366,99.784897,94.168259,94.168259,95.209700,53.658542,-4.600483,-3.372167,0.254453,-0.880508,...,-2.613855,0.389808,0.5823,-0.99309,-46.078458,3.891234,45.783766,1.537833,-0.046005,0.012875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,129.500000,131.399994,129.309998,131.350006,126.922471,100.000000,1.624763,2.377243,1.774375,2.537080,...,-0.452284,0.859787,0.701564,0.487151,-0.573913,2.165449,131.060998,1.639551,0.016248,0.007404
2022-03-28,130.820007,131.500000,129.600006,131.470001,127.184762,100.000000,0.091355,1.717602,2.470770,1.867351,...,-0.240391,0.863068,0.725094,0.505181,-0.340508,2.130055,121.945084,1.982885,0.000914,0.016248
2022-03-29,132.039993,132.839996,130.429993,131.940002,127.762773,100.000000,0.357497,0.449179,2.081240,2.837100,...,-0.026521,0.84464,0.723586,0.567238,-8.86694,2.167381,134.478125,2.355962,0.003575,0.000914


# Trying to add also PCA (Feature augmentation)

In [5]:
y = data["Up down"]
normalized = ind.normalize(data.drop(columns=["Up down", "Differenced"]))
X = normalized


In [6]:
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(X)
data["principalDf"] = principalComponents

# Save data

In [7]:
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data.to_csv("data/"+stock+".csv")

# Feature selection with GANA

In [8]:
#Remove the first 30 index which could have a value 'N'
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data=data.drop(['High','Low'], axis=1)
data

,Open,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,ROC_5,ROC_6,...,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced,principalDf
Date,,,,,,,,,,,,,,,,,,,,,
2001-02-13,109.130020,108.747612,108.639942,52.409728,-1.000870,1.562502,-0.306743,-2.702929,-0.385320,1.363393,...,0.347814,0.46947,0.9275,-49.375873,4.137277,50.383877,6.051655,-0.010009,0.025893,-2.713330
2001-02-14,108.747612,110.038239,108.933176,72.740985,1.186809,0.174060,2.767855,0.876425,-1.548198,0.796916,...,0.521018,0.491992,0.949414,-34.39673,4.053454,59.36049,5.269280,0.011868,-0.010009,-4.272781
2001-02-15,111.663483,111.644356,109.279960,98.042171,1.459599,2.663731,1.636200,4.267854,2.348817,-0.111197,...,0.510971,0.452968,0.840093,-19.914386,3.964236,125.102164,4.392343,0.014596,0.011868,-5.531401
2001-02-16,109.512428,109.942642,109.334397,71.235055,-1.524227,-0.086876,1.098902,0.087034,2.678575,0.788788,...,0.588397,0.387234,0.829511,-38.972124,3.797685,50.684093,3.838493,-0.015242,0.014596,-2.421068
2001-02-20,109.512428,106.596558,109.133110,18.524141,-3.043482,-4.521320,-3.127714,-1.978024,-2.959097,-0.446429,...,0.402078,0.267054,0.714361,-76.445413,3.905732,-23.55029,2.980371,-0.030435,-0.015242,2.895888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,129.080002,128.300003,126.476555,85.156343,-0.588869,0.156126,-0.357247,0.265711,0.991815,2.117163,...,0.609705,0.606408,0.368461,-20.425558,2.259801,93.954533,0.971475,-0.005889,0.007494,-1.588034
2022-03-24,128.330002,129.250000,126.652805,100.000000,0.740450,0.147220,0.897731,0.380557,1.008128,1.739609,...,0.786216,0.61517,0.37253,-6.95043,2.167827,93.469804,1.295513,0.007404,-0.005889,-2.986045
2022-03-25,129.500000,131.350006,126.922471,100.000000,1.624763,2.377243,1.774375,2.537080,2.011503,2.649271,...,0.859787,0.701564,0.487151,-0.573913,2.165449,131.060998,1.639551,0.016248,0.007404,-5.610019


In [9]:
y = data["Up down"]
y = y.astype('int')
#Normalize the data that we have filtered
normalized = ind.normalize(data.drop(columns=["Up down","Differenced"]))
X = normalized


In [10]:
random.seed(42)
np.random.seed(42)

estimator = DecisionTreeClassifier()
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=500,
        crossover_proba=0.5,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.5,
        mutation_independent_proba=0.15,
        tournament_size=4,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	500   	[  0.496089  15.436      0.011043]	[ 0.007548  8.682966  0.005466]	[ 0.474505  1.        0.000134]	[  0.517781  30.         0.026808]
1  	299   	[-59.503238  14.596     60.011165]	[ 772.307978    7.854475  772.268516]	[-10000.            1.            0.000479]	[     0.518154     31.        10000.      ]
2  	262   	[-19.497539  13.372     20.010324]	[ 446.788651    7.013531  446.765696]	[-10000.            1.            0.000479]	[     0.518154     32.        10000.      ]
3  	293   	[-19.496901  13.156     20.010266]	[ 446.78868     6.883289  446.765699]	[-10000.            0.            0.000479]	[     0.519662     30.        10000.      ]
4  	282   	[-39.496224  12.878     40.009809]	[ 631.221279    7.106836  631.188732]	[-10000.            0.            0.001089]	[     0.519662     28. 

In [11]:
random.seed(42)
np.random.seed(42)

estimator = linear_model.LogisticRegression(C=1e-05, penalty='none')
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=1000,
        crossover_proba=0.10,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.05,
        mutation_independent_proba=0.05,
        tournament_size=5,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	1000  	[  0.503109  15.348      0.017787]	[ 0.006106  8.652104  0.004954]	[ 0.482027  1.        0.000931]	[  0.519842  30.         0.032691]
1  	248   	[-9.491026  9.76     10.016179]   	[ 316.085716    6.905679  316.069101]	[-10000.            1.            0.000931]	[     0.520407     32.        10000.      ]
2  	276   	[ 0.513223  7.207     0.015407]   	[ 0.003803  5.490187  0.007055]      	[ 0.494256  1.        0.000931]            	[  0.523796  30.         0.032866]         
3  	289   	[-9.4844    5.864    10.017325]   	[ 316.085925    3.14603   316.069065]	[-10000.            0.            0.000931]	[     0.523796     23.        10000.      ]
4  	324   	[ 0.51816   7.073     0.022543]   	[ 0.003627  1.998417  0.006472]      	[ 0.501782  1.        0.001634]            	[  0.525864  17.       